In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import re
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [2]:
chromedriver = f'/Users/itachi/Downloads/Chrome/chromedriver'
browser = webdriver.Chrome(executable_path=chromedriver)

In [3]:
browser.get('https://tools.nycenet.edu/snapshot/2019/')

In [19]:
text_box = browser.find_element_by_tag_name('input')
text_box.click()

In [14]:
lst = browser.find_element_by_tag_name('ul')

In [17]:
schools = lst.text.split('\n')

In [18]:
schools[:10]

['P.S. 015 Roberto Clemente - [ES] 01M015',
 'P.S. 015 Roberto Clemente - [Pre-K] 01M015',
 'P.S. 019 Asher Levy - [ES] 01M019',
 'P.S. 019 Asher Levy - [Pre-K] 01M019',
 'P.S. 020 Anna Silver - [ES] 01M020',
 'P.S. 020 Anna Silver - [Pre-K] 01M020',
 'P.S. 034 Franklin D. Roosevelt - [K-8] 01M034',
 'P.S. 034 Franklin D. Roosevelt - [Pre-K] 01M034',
 'The STAR Academy - P.S.63 - [ES] 01M063',
 'The STAR Academy - P.S.63 - [Pre-K] 01M063']

In [20]:
text_box.send_keys(schools[0])

In [21]:
text_box.send_keys(Keys.ENTER)